In [1]:
#import libraries needed
from snowflake.snowpark.session import Session
import os

#check to ensure you are in your virtual Python 3.8 env
print (os.environ['CONDA_DEFAULT_ENV'])

#config to connect to Snowflake Instance
snowflake_param_dict = {
    'account': 'FLB08246',   #update with your Snowflake info
    'user': 'TAPIR',       #update with your Snowflake info         
    'password': 'TSengalla01!',    #update with your Snowflake info   
    'role': 'ACCOUNTADMIN', 
    'warehouse': 'te4437_WH',  #update with your Snowflake info
    'database': 'te4437_DW',    #update with your Snowflake info
    'schema': 'CAPSTONE_ELT_STAGE'}

#initiate the session with config (I got an authentication I had to approve)
session = Session.builder.configs(snowflake_param_dict).create()

py38_env


In [2]:
#call session.table to save tables as Snowflake Dataframe

ath_events_df = session.table('TE4437_DW.CAPSTONE_ELT_STAGE.ATHLETE_EVENTS_DW')
country_def_df = session.table('TE4437_DW.CAPSTONE_ELT_STAGE.COUNTRY_DEFINITIONS_DW')


In [3]:
#call .show() to visualize data in the Snowflake Dataframe
ath_events_df.show(5)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"NAME"                    |"SEX"  |"AGE"  |"HEIGHT"  |"WEIGHT"  |"TEAM"          |"NOC"  |"GAMES"      |"YEAR"  |"SEASON"  |"CITY"     |"SPORT"        |"EVENT"                           |"MEDAL"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1     |A Dijiang                 |M      |24     |180       |80        |China           |CHN    |1992 Summer  |1992    |Summer    |Barcelona  |Basketball     |Basketball Men's Basketball       |NULL     |
|2     |A Lamusi                  |M      |23     |170       |60        |China           |CHN    |2012 Summer  |2012    |Summer    |London     |Judo           |Judo Men's Extra

In [4]:
# View table we created in our silver layer 
silver_df = session.table('TE4437_DW.CAPSTONE_SILVER_LAYER.OLYMPIC_DATA_DW')
silver_df.show(5)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"NAME"                    |"SEX"  |"AGE"  |"HEIGHT"  |"WEIGHT"  |"TEAM"          |"NOC"  |"GAMES"      |"YEAR"  |"SEASON"  |"CITY"     |"SPORT"        |"EVENT"                           |"MEDAL"  |"REGION"     |"NOTES"  |"YEAR2"  |"SEASON2"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1     |A Dijiang                 |M      |24     |28        |388       |China           |CHN    |1992 Summer  |1992    |Summer    |Barcelona  |Basketball     |Basketball Men's Basketball       |NULL     |China        |NULL     |1992     |Summe

In [5]:
# prefer to work with pandas so import library
import pandas as pd
import matplotlib.pyplot as plt

df = silver_df.to_pandas()
df.head()

,ID,NAME,SEX,AGE,HEIGHT,WEIGHT,TEAM,NOC,GAMES,YEAR,SEASON,CITY,SPORT,EVENT,MEDAL,REGION,NOTES,YEAR2,SEASON2
0,1,A Dijiang,M,24.0,28.0,388.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None,China,None,1992,Summer
1,2,A Lamusi,M,23.0,26.0,291.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None,China,None,2012,Summer
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None,Denmark,None,1920,Summer
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,None,1900,Summer
4,5,Christine Jacoba Aaftink,F,21.0,29.0,399.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None,Netherlands,None,1988,Winter


In [6]:
# Get medal count by season for each region 
df.groupby(['REGION','SEASON'])['MEDAL'].value_counts().sort_values(ascending=False)

REGION      SEASON  MEDAL 
USA         Summer  Gold      2376
                    Silver    1304
Russia      Summer  Gold      1220
USA         Summer  Bronze    1155
Germany     Summer  Gold      1074
                              ... 
Kosovo      Summer  Gold         1
Syria       Summer  Gold         1
Kyrgyzstan  Summer  Silver       1
Macedonia   Summer  Bronze       1
Zimbabwe    Summer  Bronze       1
Name: count, Length: 427, dtype: int64

In [7]:
# Male vs Female medal winners
df2 = df.dropna(subset='MEDAL')
sex_medal_df = pd.DataFrame(df2.groupby('SEX')['MEDAL'].value_counts().reset_index())
sex_medal_df

,SEX,MEDAL,count
0,F,Bronze,3756
1,F,Gold,3722
2,F,Silver,3714
3,M,Gold,9531
4,M,Bronze,9476
5,M,Silver,9342


In [8]:
sex_df = pd.DataFrame(df.groupby('YEAR')['SEX'].value_counts())
t_df = sex_df.T
t_df

YEAR  1896  1900      1904      1906      1908      1912  ...  2008        \
SEX      M     M   F     M   F     M   F     M   F     M  ...     M     F   
count  380  1901  32  1278  16  1721  11  3039  47  3944  ...  7770  5793   

YEAR   2010        2012        2014        2016        
SEX       M     F     M     F     M     F     M     F  
count  2551  1847  7087  5791  2865  2023  7450  6203  

[1 rows x 69 columns]

In [9]:
# Turn pandas df back to a snowpark df 
snowpark_df = session.create_dataframe(sex_medal_df)

# write snowpark df back to snowflake gold table 
snowpark_df.write.mode("overwrite").save_as_table("TE4437_DW.CAPSTONE_GOLD_LAYER.sex_medal_count_table")

